## Import Packages

In [ ]:
import cv2
import numpy as np
from PIL import Image
from dataloader import Trial
import os
import random

## Global Variables

In [ ]:
RANDOM_SAMPLE = False

### Helper Functions

In [130]:
def generate_path(rando : bool) -> str:
    if rando==True:
        ROOT = os.path.dirname(os.getcwd())
        DATA_DIR = os.path.join(ROOT, 'data')
        FIN_DIR = os.path.join(DATA_DIR, random.choice(list(filter(lambda x: os.path.isdir(os.path.join(DATA_DIR, x)), os.listdir(DATA_DIR)))))
        vp = os.path.join(FIN_DIR, random.choice(list(filter(lambda x: os.path.splitext(x)[1]=='.mov', os.listdir(FIN_DIR)))))
    else:
        vp = r'..\data\TH8\TH8ST17A3D250TS0_2.mov' 
        # vp = r'..\data\TH332\TH332ST17A1D1000TS0_2.mov'

    print(vp)

    return vp


In [ ]:
def save_img(img, img_name):
    cv2.imwrite(os.path.join(os.path.dirname(os.getcwd()), 'saved_imgs', img_name)+'.png', img)

## Dynamic Feature Detection
Pre-processing includes four steps
1. We need to load our *.mov* file in as our Trial object. 
2. We convert to grayscale to reduce the information to process (and remove color variation)
3. Then, in preparation to "remove" the background from the video, we need to reduce the contrast
4. Lastly we need to remove the background

In [131]:
vid_path = generate_path(False)
recording = Trial(vid_path)
recording.to_gray(inplace=True)


..\data\TH8\TH8ST17A3D250TS0_2.mov


In [132]:
recording.change_contrast(alpha=0.5, beta=2, inplace=True)
recording.remove_avg(skipframes=100)

In [133]:
recording.play_video()

Here, we will create a copy (because we were too lazy to create an undo feature) to act as a checkpoint. We can optionally use a morphological transformation to try to fill out the air pockets as much as possible. We also check our work here at the end of this. We now have our mask for our dynamic components

In [134]:
out:Trial = recording.ranger(minval=100, maxval=240)
out.closing(inplace=True, ksize=(5,5), iterations=1)

### Error Checking

We can now check our mask

In [135]:
out.play_video()

To ensure the accuracy of our mask, we will use one of the best, most robust vision systems: the human eye. We will overlay our mask to highlight the masked areas to a brighter level than the non-masked area. This is not super effective for spotting holes in bubble masks, but it is great for spotting missed bubbles.

In [136]:
rec1 = Trial(vid_path)
rec1.apply_mask(out, inplace=True)
rec1.play_video()

In [143]:
Trial.saveImg(rec1.frames[370], "12dynamic")

## Waterline Detection

In [84]:
# vid_path = generate_path(rando=False)
recording = Trial(vid_path)
# recording = Trial(r'c:\Users\wing_\Code\ComputerVision\HMC-Computer-Vision-Project\data\TH32\TH32ST50A3D1000TS0_5.mov')
recording.to_gray(inplace=True)


In [85]:
background = recording.get_avg()


In [86]:
Trial.saveImg(background, "11background")

In [87]:
background = cv2.GaussianBlur(background, (1111, 1), sigmaX=32000, borderType=cv2.BORDER_REFLECT)

In [88]:
Trial.saveImg(background, "11smeared")

In [ ]:
Trial.show(background)

In [89]:
Z = np.float32(background.reshape((-1, 3)))

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
_, labels, centers = cv2.kmeans(Z, 3, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

centers = np.uint8(centers)
quantized = centers[labels.flatten()]
quantized = quantized.reshape(background.shape)

In [90]:
Trial.show(quantized)

In [91]:
Trial.saveImg(quantized, "11quantized")

In [ ]:
# Trial.saveImg(quantized, "quantized")

In [92]:
Trial.show(recording.frames[0])

In [ ]:
Trial.show(background)

In [93]:
yMin = quantized.shape[1]//4
darkestColor = np.unique(quantized).min()

half_mask:np.ndarray = np.ones_like(quantized)
half_mask[:yMin, :] = 0

In [94]:
waterline = 255*np.where(quantized[:, ] == darkestColor, half_mask, np.zeros_like(recording.frames[0]))

In [95]:
Trial.show(waterline)

In [96]:
Trial.saveImg(waterline, "11 waterline")

In [97]:
rec2 = Trial(vid_path)
temp = Trial(vid_path)
temp.replace(img=waterline, inplace=True)
rec2.apply_mask(temp, inplace=True)
print("Done!")

Done!


In [99]:
rec2.play_video()

In [104]:
Trial.saveImg(rec2.frames[200], "12Control")

In [ ]:
frameS = temp.frames[0]
frameO = out.frames[0]

In [ ]:
# Trial.show(frameS)

In [ ]:
# Trial.show(frameO)

In [ ]:
# maskS = np.bitwise_and(frameS, np.ones_like(frameS))
# maskO = np.bitwise_and(frameO, np.ones_like(frameS))

In [ ]:
maskS = np.bitwise_and(frameS, np.ones_like(frameS))
maskO = np.bitwise_and(frameO, np.ones_like(frameS))
multi_mask = (127.5*np.where(maskS==1, 2*maskS, maskO)).astype('uint8')

In [ ]:
# multi_mask = (127/5*np.where(maskS==1, 2*maskS, maskO)).astype('uint8')

In [ ]:
# Trial.show(multi_mask)

In [105]:
temp.combine_masks(out, inplace=True);

In [ ]:
temp.play_video()

In [106]:
Trial.show(temp.frames[200])

In [109]:
final_frame = temp.frames[200]

In [112]:
Trial.saveImg(2*final_frame, "11final_butNotBounded")

In [113]:
r0 = 0
max_bright = final_frame.max()
for r, row in enumerate(final_frame):
    if row.max() == max_bright:
        r0 = r
        break

print(r0)

520


In [114]:
splash_mask = np.ones_like(final_frame)
splash_mask[:r0, :] = 0

In [116]:
final_final_frame = 2*np.where(splash_mask, final_frame, np.zeros_like(final_frame))

In [117]:
Trial.show(final_final_frame)

In [119]:
Trial.saveImg(final_final_frame, "11FINALLY")